In [31]:
import pandas as pd
import matplotlib.pyplot as plt
import spacy
nlp = spacy.load('en_core_web_md')
from sklearn.cluster import AgglomerativeClustering
import numpy as np

In [5]:
# Load in data 
file_path_raw = './data/compas-scores-raw.csv'
file_path_non_violent = './data/compas-scores-two-years.csv'
file_path_violent = './data/compas-scores-two-years-violent.csv'

compas_data_raw = pd.read_csv(file_path_raw)
two_years_data = pd.read_csv(file_path_non_violent)
two_years_violent_data = pd.read_csv(file_path_violent)
compas_data_info = compas_data_raw.info()
compas_data_head = compas_data_raw.head()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60843 entries, 0 to 60842
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Person_ID                60843 non-null  int64  
 1   AssessmentID             60843 non-null  int64  
 2   Case_ID                  60843 non-null  int64  
 3   Agency_Text              60843 non-null  object 
 4   LastName                 60843 non-null  object 
 5   FirstName                60843 non-null  object 
 6   MiddleName               15624 non-null  object 
 7   Sex_Code_Text            60843 non-null  object 
 8   Ethnic_Code_Text         60843 non-null  object 
 9   DateOfBirth              60843 non-null  object 
 10  ScaleSet_ID              60843 non-null  int64  
 11  ScaleSet                 60843 non-null  object 
 12  AssessmentReason         60843 non-null  object 
 13  Language                 60843 non-null  object 
 14  LegalStatus           

In [6]:
# Cleaning for raw data set
# one-hot-encodes sex, ethnicity, language and maritial status
# flattens DisplayText Column to get compas scores for each individual
# Drops all other columns not of interest

def collapse_raw_data(df):
    '''
    output:
    dataframe with columns: 
    'FirstName', str
    'LastName', str
    'DateOfBirth', timestamp
    'Sex_Female', bool
    'Sex_Male', bool
    'Ethnicity_African-Am', bool
    'Ethnicity_African-American',bool
    'Ethnicity_Arabic', bool
    'Ethnicity_Asian', bool
    'Ethnicity_Caucasian',bool
    'Ethnicity_Hispanic', bool
    'Ethnicity_Native American'bool, 
    'Ethnicity_Oriental',bool
    'Ethnicity_Other', bool
    'Language_English', bool
    'Language_Spanish',bool
    'MaritalStatus_Divorced', bool
    'MaritalStatus_Married',bool
    'MaritalStatus_Separated', bool
    'MaritalStatus_Significant Other',bool
    'MaritalStatus_Single', bool
    'MaritalStatus_Unknown',bool
    'MaritalStatus_Widowed', bool
    'DecileScore_Risk of Failure to Appear', int
    'DecileScore_Risk of Recidivism', int
    'DecileScore_Risk of Violence', int
    'RawScore_Risk of Failure to Appear', float
    'RawScore_Risk of Recidivism',float
    'RawScore_Risk of Violence'float
    '''
    
    # sex_dummies = pd.get_dummies(df['Sex_Code_Text'], prefix='Sex').astype(int)
    # ethnic_dummies = pd.get_dummies(df['Ethnic_Code_Text'], prefix='Ethnicity').astype(int)
    language_dummies = pd.get_dummies(df['Language'], prefix='Language').astype(int)
    marital_dummies = pd.get_dummies(df['MaritalStatus'], prefix='MaritalStatus').astype(int)

    pivoted = df.pivot_table(
        index=['FirstName', 'LastName', 'DateOfBirth'],
        columns='DisplayText',
        values=['RawScore', 'DecileScore'],
        aggfunc='first'
    )
    pivoted.columns = ['_'.join(col).strip() for col in pivoted.columns.values]
    # df['Ethnicity_African_American'] = df[['Ethnicity_African-Am', 'Ethnicity_African-American']].max(axis=1)
    # df['Ethnicity_Asian'] = df[['Ethnicity_Asian', 'Ethnicity_Oriental']].max(axis=1)

    collapsed = df[['FirstName', 'LastName', 'DateOfBirth']].drop_duplicates()
    # collapsed = collapsed.merge(sex_dummies, left_index=True, right_index=True, how='left')
    # collapsed = collapsed.merge(ethnic_dummies, left_index=True, right_index=True, how='left')
    collapsed = collapsed.merge(language_dummies, left_index=True, right_index=True, how='left')
    collapsed = collapsed.merge(marital_dummies, left_index=True, right_index=True, how='left')
    collapsed = collapsed.merge(pivoted, left_on=['FirstName', 'LastName', 'DateOfBirth'], right_index=True, how='left')

    
    return collapsed

In [7]:
# Ensure uniqueness for FirstName, lastname and DOB
raw_data_cleaned = collapse_raw_data(compas_data_raw)
raw_data_cleaned = raw_data_cleaned.drop_duplicates(subset=['FirstName', 'LastName', 'DateOfBirth'], keep='first')
raw_data_cleaned.head(5)
raw_data_cleaned.columns

Index(['FirstName', 'LastName', 'DateOfBirth', 'Language_English',
       'Language_Spanish', 'MaritalStatus_Divorced', 'MaritalStatus_Married',
       'MaritalStatus_Separated', 'MaritalStatus_Significant Other',
       'MaritalStatus_Single', 'MaritalStatus_Unknown',
       'MaritalStatus_Widowed', 'DecileScore_Risk of Failure to Appear',
       'DecileScore_Risk of Recidivism', 'DecileScore_Risk of Violence',
       'RawScore_Risk of Failure to Appear', 'RawScore_Risk of Recidivism',
       'RawScore_Risk of Violence'],
      dtype='object')

In [8]:
def process_compas_data(df):
    '''
    output:
    'id', int
    'FirstName', str
    'LastName', str
    'DateOfBirth', timestamp
    'priors_count', int
    'age',int
    'v_decile_score', int
    'two_year_recid', bool
    'is_recid', bool
    'juv_fel_count',bool
    'juv_misd_count', bool
    'juv_other_count', bool
    'c_charge_degree', cat
    'c_charge_desc',cat
    'above_45', bool
    'below_25'bool
    
    '''
    columns_of_interest = [
        'id', 'first', 'last', 'dob', 'priors_count', 'age',
        'v_decile_score', 'two_year_recid', 'is_recid',
        'juv_fel_count', 'juv_misd_count', 'juv_other_count',
        'c_charge_degree', 'c_charge_desc', 'race','sex'
    ]
    df = df[columns_of_interest]

    df['above_45'] = (df['age'] > 45).astype(int)
    df['below_25'] = (df['age'] < 25).astype(int)
    df['is_felon'] = (df['c_charge_degree'] =='F').astype(int)
    df['is_misdemeanor'] = (df['c_charge_degree'] =='M').astype(int)
    df = df.drop(columns=['c_charge_degree'])
    sex_dummies = pd.get_dummies(df['sex']).astype(int)
    ethnic_dummies = pd.get_dummies(df['race']).astype(int)
    df = df.merge(sex_dummies, left_index=True, right_index=True, how='left')
    df = df.merge(ethnic_dummies, left_index=True, right_index=True, how='left')
    # charge_degree_dummies = pd.get_dummies(df['c_charge_degree'], prefix='ChargeDegree').astype(int)
    # charge_desc_dummies = pd.get_dummies(df['c_charge_desc'], prefix='ChargeDesc').astype(int)
    # df = df.merge(charge_desc_dummies, left_index=True, right_index=True, how='left')
    # df = pd.concat([df, charge_degree_dummies, charge_desc_dummies], axis=1)

    # df.drop(columns=['c_charge_degree', 'c_charge_desc'], inplace=True)
    df.rename(columns={'first': 'FirstName', 'last': 'LastName', 'dob': 'DateOfBirth'}, inplace=True)
    return df

In [9]:
# Join raw data with compas-scores-two-years/violent
# due to common index / id joining on first name, last name and DOB
# After manual inspection this appears to work OK

def normalize_and_join(raw_df, compas_df):
    raw_df['FirstName'] = raw_df['FirstName'].str.strip().str.lower()
    raw_df['LastName'] = raw_df['LastName'].str.strip().str.lower()
    raw_df['DateOfBirth'] = pd.to_datetime(raw_df['DateOfBirth'], errors='coerce')
    raw_df = raw_df.drop_duplicates(subset=['FirstName', 'LastName', 'DateOfBirth'], keep='first')

    compas_df['FirstName'] = compas_df['FirstName'].str.strip().str.lower()
    compas_df['LastName'] = compas_df['LastName'].str.strip().str.lower()
    compas_df['DateOfBirth'] = pd.to_datetime(compas_df['DateOfBirth'], errors='coerce')
    combined_df_all = pd.merge(
        raw_df,
        compas_df,
        on=['FirstName', 'LastName', 'DateOfBirth'],
        how='outer',
        indicator=True
    )

    combined_df_matches = pd.merge(
        compas_df,
        raw_df,
        on=['FirstName', 'LastName', 'DateOfBirth'],
        how='left',
        
        indicator=True
    )
    combined_df_all['Enhanced_Data'] = (combined_df_all['_merge'] == 'both').astype(int)
    combined_df_all.drop(columns=['_merge'], inplace=True)
    duplicates = raw_df[raw_df.duplicated(subset=['FirstName', 'LastName', 'DateOfBirth'], keep=False)]
    return combined_df_all, combined_df_matches, raw_df


In [10]:
def test_data_cleaning(raw_data, compas_two_years_data, compas_two_years_violent_data):
    # TODO
    # Write test cases for data cleaning 
    pass

In [11]:
# Helper function to test for bad merge results between compas-scores-two-years datasets
def check_collisions(df, columns_to_check, suffixes=('_general', '_violent')):
    collision_flag = pd.Series(0, index=df.index)  
    collision_details = []

    for col in columns_to_check:
        col_general = f"{col}{suffixes[0]}"
        col_violent = f"{col}{suffixes[1]}"
        
        if col_general in df.columns and col_violent in df.columns:
            mismatch = df[col_general] != df[col_violent]
            mismatch = mismatch & (~df[col_general].isnull()) & (~df[col_violent].isnull())  # Ignore NaNs
            collision_flag |= mismatch.astype(int)
            collision_details.append((col, df[mismatch][[col_general, col_violent]]))

    df['Collision_Flag'] = collision_flag

    for col, mismatches in collision_details:
        print(f"Collisions detected in column: {col}")
        print(mismatches)

    return df


In [12]:
two_years_data_processed = process_compas_data(two_years_data)
two_years_violent_data_processed = process_compas_data(two_years_violent_data)


/var/folders/jh/g1jlvxkd1nv5yvt4q5pmbzn00000gn/T/ipykernel_47243/1907432612.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['above_45'] = (df['age'] > 45).astype(int)
/var/folders/jh/g1jlvxkd1nv5yvt4q5pmbzn00000gn/T/ipykernel_47243/1907432612.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['below_25'] = (df['age'] < 25).astype(int)
/var/folders/jh/g1jlvxkd1nv5yvt4q5pmbzn00000gn/T/ipykernel_47243/1907432612.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

In [13]:
two_years_violent_data_processed.head(5)

,id,FirstName,LastName,DateOfBirth,priors_count,age,v_decile_score,two_year_recid,is_recid,juv_fel_count,...,is_felon,is_misdemeanor,Female,Male,African-American,Asian,Caucasian,Hispanic,Native American,Other
0,1,miguel,hernandez,1947-04-18,0,69,1,0,0,0,...,1,0,0,1,0,0,0,0,0,1
1,3,kevon,dixon,1982-01-22,0,34,1,1,1,0,...,1,0,0,1,1,0,0,0,0,0
2,5,marcu,brown,1993-01-21,1,23,6,0,0,0,...,1,0,0,1,1,0,0,0,0,0
3,6,bouthy,pierrelouis,1973-01-22,2,43,1,0,0,0,...,1,0,0,1,0,0,0,0,0,1
4,7,marsha,miles,1971-08-22,0,44,1,0,0,0,...,0,1,0,1,0,0,0,0,0,1


In [14]:
two_years_data_processed.head(5)

,id,FirstName,LastName,DateOfBirth,priors_count,age,v_decile_score,two_year_recid,is_recid,juv_fel_count,...,is_felon,is_misdemeanor,Female,Male,African-American,Asian,Caucasian,Hispanic,Native American,Other
0,1,miguel,hernandez,1947-04-18,0,69,1,0,0,0,...,1,0,0,1,0,0,0,0,0,1
1,3,kevon,dixon,1982-01-22,0,34,1,1,1,0,...,1,0,0,1,1,0,0,0,0,0
2,4,ed,philo,1991-05-14,4,24,3,1,1,0,...,1,0,0,1,1,0,0,0,0,0
3,5,marcu,brown,1993-01-21,1,23,6,0,0,0,...,1,0,0,1,1,0,0,0,0,0
4,6,bouthy,pierrelouis,1973-01-22,2,43,1,0,0,0,...,1,0,0,1,0,0,0,0,0,1


In [15]:
# temp merge to check for collisions 
compas_combined = pd.merge(
    two_years_data_processed,
    two_years_violent_data_processed,
    on='id',
    how='outer',
    suffixes=('_general', '_violent')
)


In [16]:
# Sanity check merging data is safe and we don't have any collisions 
# Results look good, we can proceed
columns_to_check = ['age', 'priors_count', 'juv_fel_count', 'v_decile_score', 'two_year_recid', 'Female', 'Male', 'African-American', 'Caucasian', 'Asian', 'Hispanic']
compas_combined = check_collisions(compas_combined, columns_to_check)


Collisions detected in column: age
Empty DataFrame
Columns: [age_general, age_violent]
Index: []
Collisions detected in column: priors_count
Empty DataFrame
Columns: [priors_count_general, priors_count_violent]
Index: []
Collisions detected in column: juv_fel_count
Empty DataFrame
Columns: [juv_fel_count_general, juv_fel_count_violent]
Index: []
Collisions detected in column: v_decile_score
Empty DataFrame
Columns: [v_decile_score_general, v_decile_score_violent]
Index: []
Collisions detected in column: two_year_recid
Empty DataFrame
Columns: [two_year_recid_general, two_year_recid_violent]
Index: []
Collisions detected in column: Female
Empty DataFrame
Columns: [Female_general, Female_violent]
Index: []
Collisions detected in column: Male
Empty DataFrame
Columns: [Male_general, Male_violent]
Index: []
Collisions detected in column: African-American
Empty DataFrame
Columns: [African-American_general, African-American_violent]
Index: []
Collisions detected in column: Caucasian
Empty Dat

In [17]:
# Identify overlapping columns (excluding the key column 'id')
overlapping_columns = [col for col in two_years_violent_data_processed.columns 
                       if col in two_years_data_processed.columns and col != 'id']

# Drop overlapping columns from one dataset
two_years_violent_data_subset = two_years_violent_data_processed.drop(columns=overlapping_columns)

# Merge datasets without duplicating columns
compas_combined = pd.merge(
    two_years_data_processed,
    two_years_violent_data_subset,
    on='id',
    how='outer'
)
compas_combined.head(5)
compas_combined = compas_combined.dropna(subset=['FirstName', 'LastName'])


In [18]:
print(len(two_years_data_processed))
print(len(two_years_violent_data_processed))
print(len(compas_combined))
print(len(raw_data_cleaned))
duplicates = raw_data_cleaned[raw_data_cleaned.duplicated(subset=['FirstName', 'LastName', 'DateOfBirth'], keep=False)]
print(duplicates)
# Should be: 
#7214
#4743
#7219
#18523

7214
4743
7214
18523
Empty DataFrame
Columns: [FirstName, LastName, DateOfBirth, Language_English, Language_Spanish, MaritalStatus_Divorced, MaritalStatus_Married, MaritalStatus_Separated, MaritalStatus_Significant Other, MaritalStatus_Single, MaritalStatus_Unknown, MaritalStatus_Widowed, DecileScore_Risk of Failure to Appear, DecileScore_Risk of Recidivism, DecileScore_Risk of Violence, RawScore_Risk of Failure to Appear, RawScore_Risk of Recidivism, RawScore_Risk of Violence]
Index: []


In [19]:

combined_df_all, combined_df_matches, raw_data_cleaned = normalize_and_join(raw_data_cleaned, compas_combined)
print(len(combined_df_all))
print(len(combined_df_matches))
duplicates = combined_df_matches[combined_df_matches.duplicated(subset=['FirstName', 'LastName', 'DateOfBirth'], keep=False)]
print(duplicates)





/var/folders/jh/g1jlvxkd1nv5yvt4q5pmbzn00000gn/T/ipykernel_47243/3230235621.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  raw_df['DateOfBirth'] = pd.to_datetime(raw_df['DateOfBirth'], errors='coerce')


20542
7214
Empty DataFrame
Columns: [id, FirstName, LastName, DateOfBirth, priors_count, age, v_decile_score, two_year_recid, is_recid, juv_fel_count, juv_misd_count, juv_other_count, c_charge_desc, race, sex, above_45, below_25, is_felon, is_misdemeanor, Female, Male, African-American, Asian, Caucasian, Hispanic, Native American, Other, Language_English, Language_Spanish, MaritalStatus_Divorced, MaritalStatus_Married, MaritalStatus_Separated, MaritalStatus_Significant Other, MaritalStatus_Single, MaritalStatus_Unknown, MaritalStatus_Widowed, DecileScore_Risk of Failure to Appear, DecileScore_Risk of Recidivism, DecileScore_Risk of Violence, RawScore_Risk of Failure to Appear, RawScore_Risk of Recidivism, RawScore_Risk of Violence, _merge]
Index: []

[0 rows x 43 columns]


In [20]:
combined_df_all.head(5)

,FirstName,LastName,DateOfBirth,Language_English,Language_Spanish,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Separated,MaritalStatus_Significant Other,MaritalStatus_Single,...,is_misdemeanor,Female,Male,African-American,Asian,Caucasian,Hispanic,Native American,Other,Enhanced_Data
0,aajah,herrington,1992-06-02,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1
1,aaliyah,lovo,1994-11-10,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1
2,aaron,aprile,1988-11-02,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,aaron,brinson,1985-07-17,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,aaron,cabrera,1987-03-27,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [21]:
combined_df_all.columns

Index(['FirstName', 'LastName', 'DateOfBirth', 'Language_English',
       'Language_Spanish', 'MaritalStatus_Divorced', 'MaritalStatus_Married',
       'MaritalStatus_Separated', 'MaritalStatus_Significant Other',
       'MaritalStatus_Single', 'MaritalStatus_Unknown',
       'MaritalStatus_Widowed', 'DecileScore_Risk of Failure to Appear',
       'DecileScore_Risk of Recidivism', 'DecileScore_Risk of Violence',
       'RawScore_Risk of Failure to Appear', 'RawScore_Risk of Recidivism',
       'RawScore_Risk of Violence', 'id', 'priors_count', 'age',
       'v_decile_score', 'two_year_recid', 'is_recid', 'juv_fel_count',
       'juv_misd_count', 'juv_other_count', 'c_charge_desc', 'race', 'sex',
       'above_45', 'below_25', 'is_felon', 'is_misdemeanor', 'Female', 'Male',
       'African-American', 'Asian', 'Caucasian', 'Hispanic', 'Native American',
       'Other', 'Enhanced_Data'],
      dtype='object')

In [22]:
combined_df_matches.head(5)

,id,FirstName,LastName,DateOfBirth,priors_count,age,v_decile_score,two_year_recid,is_recid,juv_fel_count,...,MaritalStatus_Single,MaritalStatus_Unknown,MaritalStatus_Widowed,DecileScore_Risk of Failure to Appear,DecileScore_Risk of Recidivism,DecileScore_Risk of Violence,RawScore_Risk of Failure to Appear,RawScore_Risk of Recidivism,RawScore_Risk of Violence,_merge
0,1,miguel,hernandez,1947-04-18,0.0,69.0,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,3,kevon,dixon,1982-01-22,0.0,34.0,1.0,1.0,1.0,0.0,...,1.0,0.0,0.0,6.0,3.0,1.0,25.0,-0.76,-3.07,both
2,4,ed,philo,1991-05-14,4.0,24.0,3.0,1.0,1.0,0.0,...,1.0,0.0,0.0,7.0,4.0,3.0,26.0,-0.66,-2.26,both
3,5,marcu,brown,1993-01-21,1.0,23.0,6.0,0.0,0.0,0.0,...,1.0,0.0,0.0,3.0,8.0,6.0,19.0,0.16,-1.59,both
4,6,bouthy,pierrelouis,1973-01-22,2.0,43.0,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [23]:
combined_df_matches.columns

Index(['id', 'FirstName', 'LastName', 'DateOfBirth', 'priors_count', 'age',
       'v_decile_score', 'two_year_recid', 'is_recid', 'juv_fel_count',
       'juv_misd_count', 'juv_other_count', 'c_charge_desc', 'race', 'sex',
       'above_45', 'below_25', 'is_felon', 'is_misdemeanor', 'Female', 'Male',
       'African-American', 'Asian', 'Caucasian', 'Hispanic', 'Native American',
       'Other', 'Language_English', 'Language_Spanish',
       'MaritalStatus_Divorced', 'MaritalStatus_Married',
       'MaritalStatus_Separated', 'MaritalStatus_Significant Other',
       'MaritalStatus_Single', 'MaritalStatus_Unknown',
       'MaritalStatus_Widowed', 'DecileScore_Risk of Failure to Appear',
       'DecileScore_Risk of Recidivism', 'DecileScore_Risk of Violence',
       'RawScore_Risk of Failure to Appear', 'RawScore_Risk of Recidivism',
       'RawScore_Risk of Violence', '_merge'],
      dtype='object')

In [25]:
combined_df_matches.c_charge_desc.head(5)

0      Aggravated Assault w/Firearm
1    Felony Battery w/Prior Convict
2             Possession of Cocaine
3            Possession of Cannabis
4             arrest case no charge
Name: c_charge_desc, dtype: object

In [29]:
combined_df_matches = combined_df_matches.dropna(subset=['c_charge_desc'])
combined_df_matches['embeddings'] = combined_df_matches['c_charge_desc'].apply(lambda x: nlp(x).vector)


In [ ]:
embedding_matrix = np.stack(combined_df_matches['embeddings'].to_numpy())
clustering = AgglomerativeClustering(n_clusters=10).fit(embedding_matrix)
combined_df_matches['aggregated_category'] = clustering.labels_


In [36]:
combined_df_matches.to_csv('./compas_data_combined_matches_dim.csv', index=False)
#combined_df_all.to_csv('./compas_data_combined_full.csv', index=False)

